# Area statistics for protected areas in mainland Norway

[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ac-willeke/pygdal-geo-engineer/blob/main/notebooks/2024.01_conservation_and_preservation.ipynb) [![github](https://img.shields.io/badge/GitHub-View%20on%20GitHub-blue?logo=github)](https://github.com/ac-willeke/pygdal-geo-engineer/blob/main/notebooks/2024.01_conservation_and_preservation.ipynb)

**Author**: Willeke A'Campo

**Description:** This notebook is designed to compute area statistics for protected regions within mainland Norway. It specifically targets two datasets: **Verneområder** og **Foreslatte verneområder**, both provided by the Norwegian Environment Agency. In addition area statistics are calcualted for the whole of norway and for each regional zone. This allows for a comparative analysis beteween the protected areas and the overal Norwegian Territory. 

The attributes calculated can be divided into three groups:

1. Area variables for the protected areas:
    - Area (terrestrial and marine)
    - Perimeter (terrestrial and marine)
    - Land area (terrestrial)
    - Perimeter land area (terrestrial)

2. Overlay statistics for the protected area and the following datasets:
    - Infrastrukturindeks (raster overlay)
        - Mean value of infrastructure index per protected area 
    - AR50 - Bonitet (vector overlay)
        - Area proportion of land cover quality classes 
    - Bioklimatiske soner (vector overlay)
        - Area proportion of bioclimatic zone classes

3. Spatial indices for the protected areas:
    - Shape Index for each protected area (land + marine)
    - Shape index for each protected area (land)
    - Density of protected area per 10x10km (SSB grid)






# 0. Configure Notebook

In [1]:
# Variables to control the execution of the workflow
gis_server = True # True = use GIS server, False = use local files
load_data = False # True = load data from source, False = data is already loaded
test_area = False # True = test area, False = full area
prepare_data = True
prepare_duckdb = True # True = prepare duckdb, False = duckdb is already prepared
run_infra = True
run_ar50 = True # True = run AR50 area calc , False = AR50 is already run
run_bioklima = True 
run_shape_index = True

# Set the statistical zone: foreslatt_vern, naturvernomrade, norge_landareal_n50, region_2024
statistical_zone = "foreslatt_vern"

if statistical_zone == "foreslatt_vern":
    sz_land = "f_vern_land"
    sz_sea = "f_vern_sjo"
    sz_land_sea = "f_vern_land_sjo"
    sz_land_vars = "foreslatt_vern_overlapp"
    
if statistical_zone == "naturvernomrade":
    sz_land = "vern_land"
    sz_sea = "vern_sjo"
    sz_land_sea = "vern_land_sjo"
    sz_land_vars = "naturvernomrade_overlapp"


In [2]:
import os 
from osgeo import gdal
from pathlib import Path
from itertools import islice

import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import box

# mapping libraries
# https://leafmap.org/faq/#how-to-use-a-specific-plotting-backend
import leafmap.foliumap as leafmap

In [3]:
project_path= Path.cwd().parents[0]
shell_path= os.path.join(project_path, "src", "shell_scripts")
python_path= os.path.join(project_path, "src", "py_scripts")

print(f"Gdal Version: {gdal.__version__} ")
print(f"Project Path: {project_path}")
print(f"Path to shell scripts: {shell_path}")
print(f"Path to python scripts: {python_path}")

Gdal Version: 3.8.4 
Project Path: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer
Path to shell scripts: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer/src/shell_scripts
Path to python scripts: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer/src/py_scripts


In [4]:
# import local python scripts
from py_scripts.my_ogr import *
from py_scripts.my_duckdb import *
from py_scripts.raster import * 
from py_scripts.config import load_catalog, load_parameters
from py_scripts import decorators as dec

# 1. Data pre-processing

------------
**Data folder structure**

```bash
|-- data
|   |-- raw                                          # single raw files received or downloaded 
|   |-- interim
|   |    |-- vern_og_bevaring.db                     # DuckDB database with (pre-processed) data layers for spatial sql analysis (ca. 30 GB)
|   |    |-- vern_25833.gpkg                         # GPKG for viewing protected area data in GIS-software
|   |    |-- bevaring_25833.gpkg                     # GPKG for viewing environment data in GIS. 
|   |    |-- admin_25833.gpkg                        # GPKG for administrative data in GIS. 
|   |-- processed
|   |    |-- vern_og_bevaring.gpkg                   # GPKG with resulting data layers 
```
------------

<br>

**Data Pipeline**

1. Raw data is downloaded and stored into the `data\raw` folder. 
2. The raw files are converted (or if possible directly imported) into the GPKGs located in `data\interim`. These GPKGS are used to view the data in GIS-software before analysis. 
3. The data layers in the GPKG are loaded into a DuckDB database and/or a (geo)dataframe for spatial analysis. 
4. The results are exported to a GPKG located in `data\processed`. 
5. Raw and Interim files can be deleted by setting the variables: *delete_raw* and *delete_interim* to True



<br>

**GeoPackage information**

**INTERIM:**

/interim/*admin_25833.gpkg*

| Filename | Dataset Name | Description | Year | Source |
|----------| ------------ | ----------- | ---- | ------ |
| fylke_2024 | Fylker, 2024 | Provincial Boundaries | 2024 | [SSB](https://www.ssb.no/en/kart/griddata) |
| norge_havareal_ar50 | Norge havareal | Norwegian sea area based on AR50 (Arealtype = hav) | 2022 | [NIBIO](https://kart8.nibio.no/nedlasting/dashboard)|
| norge_landareal_ar50 | Norge landareal | Norwegian land area based on AR50 (Arealtype =! hav) | 2022 | [NIBIO](https://kart8.nibio.no/nedlasting/dashboard)|
| norge_ruter_10km | SSB rutenett (10x10 km)| Grid for Norway | 2024 | [SSB](https://kart.ssb.no/) |
| region_2024 | Regioner, 2024 | Regional zones: Nord, Midt, Sør, Øst, Vest based on Provincial boundaries | 2024 |  [SSB](https://www.ssb.no/en/kart/griddata) |

/interim/*vern_25833.gpkg*

| Layer | Dataset Name | Description | Year | Source |
|-------| ------------ | ----------- | ---- | ------ |
| naturvernomrade | Naturvernområder| Nature protected areas | 2024 | [Miljødirektoratet](https://kartkatalog.miljodirektoratet.no/Dataset/Details/0) |
| foreslatt_vern | Foreslåtte naturvernområder | Planned nature protected areas | 2024 | [Miljødirektoratet](https://kartkatalog.miljodirektoratet.no/Dataset/Details/1)| 

- *naturvernomrade* and *foreslatt_vern* imported from arcgis rest server into GPKG using ogr2ogr

/interim/*bevaring_25833.gpkg*

|Layer| Dataset Name | Description | Year | Source |
|-----| ------------ | ----------- | ---- | ------ |
| ar50_flate | AR50 | Land cover classes | 2022 | [NIBIO](https://kart8.nibio.no/nedlasting/dashboard) |
| bioklima_2017_coastalclip | Bioklimatiske soner | Bioclimatic zones stored in 1x1 km grid | 2017 | [Artsdatabanken](https://data.artsdatabanken.no/Natur_i_Norge/Natursystem/Beskrivelsessystem/Regional_naturvariasjon/Bioklimatisk_sone) |
| Infra25m | Infrastrukturindeks | Infrastructure index (25m) raster converted to vector datasett. | 2024 | Internal NINA datasett |

- *ar50_2022* stored on NINA's server. Manually loaded into GPKG
- *bioklima_soner_2017* received from V. Bakkestuen (NINA) manually loaded into GPKG. 
- *Infra25m* received from V. Bakkestuen (NINA) converted to vector and lmanually oaded into GPKG.



**PROCESSED**

/processed/*vern_og_bevaring_25833.gpkg*

| Layer | Description | Year | Source |
|-------| ----------- | ---- | ------ |
| naturvernomrade | Nature protected areas containing attributes for (i) area statistics for  AR50, Bioclimatic zones, Infrastructureindex (ii) shape index. | 2024 | Project-folder, not published |
| foreslatt_vern | Planned nature protected areas containing attributes for (i) area statistics for  AR50, Bioclimatic zones, Infrastructureindex (ii) shape index. | 2024 | Project-folder, not published |
| fastland_norge | Norway mainland area containing attributes for area statistics for AR50, Bioclimatic zones, Infrastructureindex | 2024 | Project-folder, not published |
| region | Regional zones containing attributes for area statistics for AR50, Bioclimatic zones, Infrastructureindex | 2024 | Project-folder, not published |




In [5]:
# init catalog and params
catalog = load_catalog()
params = load_parameters()

# load paths from catalog
data_path = catalog["project_data"]["filepath"]
raw_data = os.path.join(data_path, "raw")
interim_data = os.path.join(data_path, "interim")

admin_gpkg = os.path.join(data_path, "interim", "admin_25833.gpkg")
vern_gpkg = os.path.join(data_path, "interim", "vern_25833.gpkg")
bevaring_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
vern_og_bevaring_gpkg = os.path.join(data_path, "interim", "vern_og_bevaring_25833.gpkg")
db_path = os.path.join(data_path, "processed", "bevaring.db")

print(f"Path to data: {data_path}")

Path to data: /home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data


## 1.1 Download Data

### Vector Data
**Import data into `admin_25833.gpkg`**

Note Norwegians's Land Area is calculated from Norways National Map N50. Please check the *norge_landareal_n50* in GIS software to see if the layer is correctly clipped! 

In [6]:
if load_data:
    
    # load paths from catalog    
    fylke_path = catalog["fylke"]["filepath"]
    fylke_lyr = catalog["fylke"]["layer"]
    ar50_path = catalog["ar50"]["filepath"]
    ar50_lyr = catalog["ar50"]["layer"]
    n50_path = catalog["n50"]["filepath"]
    n50_lyr = catalog["n50"]["layer"]

    # RUTER 10KM NORGE (SSB)
    # download from ssb, unzip, convert to gpkg, remove shapefile and zip
    ! curl -o Ruter_10KM_norge.zip "https://www.ssb.no/natur-og-miljo/_attachment/375082?_ts=1685c0e69b8"
    ! unzip Ruter_10KM_norge.zip 
    ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln norge_ruter_10km -nlt MULTIPOLYGON $admin_gpkg $raw_data/"Ruter_10km_Norge.shp" -lco "OVERWRITE=YES"
    ! find $raw_data -type f -name "Ruter_10km_Norge.*" -exec echo "Removing {}" \; -exec rm {} \;
    ! rm Ruter_10KM_norge.zip
    
    # FYLKE
    ! ogrinfo "$fylke_path"
    ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln "$fylke_lyr" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$fylke_path" "$fylke_lyr"
    
    # NORGE
    # dissolve all fylke to one feature
    ! ogr2ogr -append -f "GPKG" -dialect sqlite -sql "SELECT ST_Union(Shape) AS geom, 'Norge' AS fylkesnavn FROM $fylke_lyr" -t_srs EPSG:25833 -nln "norge" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$fylke_path"
    
    # AR50 HAVFLATE
    # import AR50 where artype = 82 and dissolve to one feature (Simplify by 1 m)
    ! ogr2ogr -append -f "GPKG" -dialect sqlite -sql "SELECT ST_Union(Shape) AS geom, artype FROM $ar50_lyr WHERE artype = '82'" -t_srs EPSG:25833 -nln "norge_havflate_ar50" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$ar50_path"
    
    # N50 HAVFLATE
    # where objtype = Havflate and dissolve to one feature
    ! ogr2ogr -append -f "GPKG" -dialect sqlite -sql "SELECT ST_Union(Shape) AS geom, objtype FROM $n50_lyr WHERE objtype = 'Havflate'" -t_srs EPSG:25833 -nln "norge_havflate_n50" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$n50_path"
    
    # N50 LANDAREAL
    remake = False
    
    if remake:
        # norge_landareal_n50: norge - norge_havflate_n50
        ! ogr2ogr -append -f "GPKG" -dialect sqlite -sql "SELECT ST_Difference(norge.geom, norge_havflate_n50.geom) AS geom, 'land' AS objtype FROM norge, norge_havflate_n50" \
            -t_srs EPSG:25833 -nln "norge_landareal_n50" -nlt MULTIPOLYGON admin_25833.gpkg admin_25833.gpkg
    else: 
        # copy layer from raw data
        ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln "norge_landareal_n50" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$raw_data/N50_2024_hav_landareal_25833.gpkg" "norge_landareal_n50"

**Import data into `vern_25833.gpkg`**

In [7]:
if load_data: 
    # download data from ArcGIS REST directly into the interim folder
    os.chdir(os.path.join(data_path, "interim"))
    print(os.getcwd())
        
    # ArcGIS REST: Naturvern (02.2024)
    ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln naturvernomrade -nlt MULTIPOLYGON vern_25833.gpkg \
        "https://kart.miljodirektoratet.no/arcgis/rest/services/vern/mapserver/0/query?where=1%3D1&outfields=*&f=json&resultRecordCount=1"

    # ArcGIS REST: Foreslåtte verneområder (02.2024)
    ! ogr2ogr -append -f "GPKG" -t_srs EPSG:25833 -nln foreslatt_vern -nlt MULTIPOLYGON vern_25833.gpkg \
        "https://kart.miljodirektoratet.no/arcgis/rest/services/vern/mapserver/4/query?where=1%3D1&outfields=*&f=json&resultRecordCount=1"

**Import data into bevaring_25833.gpkg**

In [8]:
if load_data:
    os.chdir(os.path.join(data_path, "interim"))
    print(os.getcwd())
    
    bioklima_path = catalog["bioclimatic_zones"]["filepath"]
    bioklima_lyr = catalog["bioclimatic_zones"]["layer"]
    ar50_path = catalog["ar50"]["filepath"]
    ar50_lyr = catalog["ar50"]["layer"]

    print(ar50_path, ar50_lyr)
    
    # Copy GDB layers from NINA R to gpkg
    ! ogrinfo "$bioklima_path"
    ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln soner_2017_1km -nlt MULTIPOLYGON bevaring_25833.gpkg "$bioklima_path" "$bioklima_lyr"
    ! ogrinfo "$ar50_path"
    ! ogr2ogr -append -f "GPKG" -t_srs EPSG:25833 -nln "$ar50_lyr" -nlt MULTIPOLYGON bevaring_25833.gpkg "$ar50_path" "$ar50_lyr"


### Raster Data



In [9]:
if load_data:
    infra_path = catalog["infra_25m"]["filepath"]
    
    # copy to project raw folder and convert to 25833
    ! gdalwarp -t_srs EPSG:25833 $infra_path $raw_data/infra_25m_25833.tif

In [10]:
# clip raster by polygon 
if load_data:
    ! gdalwarp -cutline "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/norge_landareal_n50.shp" -crop_to_cutline \
        $raw_data/infra_25m_25833.tif \
        $interim_data/infra_25m_25833_coastalclip.tif

In [11]:
# WARNING: run in bash shell, otherise correct path to gdal_calc.py is not found. 
# multiply all cells by 1000 except for nodata values. 
if load_data:
    ! gdal_calc.py -A "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/infra_25m_25833_coastalclip.tif" \
        --outfile="//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/infra_25m_25833_x1000.tif" \
            --calc="numpy.where(A!=3.4e+38, A*1000, 3.4e+38)" --type=Float32 --NoDataValue=3.4e+38

In [12]:
# to integer
if load_data:
    ! gdal_translate -ot Int16 -of GTiff \
        "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/infra_25m_25833_x1000.tif" \
        "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/infra_25m_25833_x1000_int.tif"

In [13]:
# clip by coast again 
if load_data:
    ! gdalwarp -cutline "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/norge_landareal_n50.shp" -crop_to_cutline \
        $interim_data/infra_25m_25833_x1000_int.tif \
        $interim_data/infra_25m_25833_x1000int_coastalclip.tif 

In [14]:
# raster to vector
# to BIG! 
if load_data:
    ! gdal_polygonize.py $interim_data/infra_25m_25833_x1000_int.tif -f "GeoJSON" $raw_data/infra_25m_25833_x1000_int.geojson

**Write Metadata to .txt**

In [15]:
if load_data: 
    # run shell scripts from /src/shell
    os.chdir(shell_path)

    path = os.path.join(data_path, "interim")

    ! gdalinfo --version
    ! chmod +x gdal_gpkg-info.sh
    ! ./gdal_gpkg-info.sh $path "vern_25833.gpkg"
    ! ./gdal_gpkg-info.sh $path "bevaring_25833.gpkg"
    ! ./gdal_gpkg-info.sh $path "admin_25833.gpkg"

## 1.2 Data Preparation

### Sudy Area | Norway Mainland

This workflow is created and run for Norway mainland. If you like to test or use the workflow we recommend running it first on a study area. Below are bounding box coordinates provided for Dovrefjell, Fosen or Trondheim area. 

**Create Bounding Box** 

In [16]:
# bounding box Dovrefjell
bounding_box = "fosen"  # "dovrefjell" or "trondheim"

if bounding_box == "dovrefjell":
    xmin, ymin, xmax, ymax = 160000.00, 6900000.00, 260000.00, 6950000.00
if bounding_box == "fosen":
    xmin, ymin, xmax, ymax = 180000.00, 7010000.00, 290000.00, 7150000.00
if bounding_box == "trondheim":
    xmin, ymin, xmax, ymax = 260520.12, 7032142.5, 278587.56, 7045245.27

# Create a bounding box
boxBB =  box(xmin, ymin, xmax, ymax)
crs = 'EPSG:25833'

gdf_BB = gpd.GeoDataFrame(geometry=[boxBB])
gdf_BB['name'] = 'Dovre_BB'
gdf_BB.crs = crs
bounds = gdf_BB.bounds.to_numpy().tolist()[0]

In [17]:
if test_area: 
    # run shell scripts from /src/shell
    os.chdir(shell_path)
    for file in ["vern", "bevaring", "admin"]:
        input_file = os.path.join(data_path, "interim", f"{file}_25833.gpkg")
        output_file = os.path.join(data_path, "tmp", f"{file}_25833_bbox.gpkg")
        
        #for layer_name in layer_names:
        ! chmod +x gdal_copy-file-bbox.sh
        ! ./gdal_copy-file-bbox.sh {input_file} {output_file} {xmin} {ymin} {xmax} {ymax}

### AR50 - Bonitet

Translate the Bonitet classes using a lookup table.
 
&emsp; $Bonitet = (artype, artreslag, arskogbon, arjordbr, ardyrkning, arveget)$ 

In [18]:
# ar50 layer
layer_name, new_field_name = "ar50_flate", "ar50_bonitet"

if test_area:
    # import gpkg into ogr object 
    # add field name if it does not exist
    tmp_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
    ds, lyr = import_gpkg(tmp_gpkg, layer_name, new_field_name)
    #print_layer_schema(lyr)
else:
    # import gpkg into ogr object 
    # add field name if it does not exist
    in_gpkg = os.path.join(interim_data, "bevaring_25833.gpkg")
    print(in_gpkg)
    ds, lyr = import_gpkg(in_gpkg, layer_name, new_field_name)
    #print_layer_schema(lyr)

# Convert the first 5 features of the layer to a DataFrame
# use islice to limit the number of features to 5, to reduce computation time
df_AR50 = pd.DataFrame(feature.items() for feature in islice(lyr, 5))

# Print the DataFrame
df_AR50

/home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/bevaring_25833.gpkg


,OBJECTID,objtype,artype,arskogbon,artreslag,arveget,arjordbr,ardyrking,arkartstd,målemetode,informasjon,oppdateringsdato,originaldatavert,navnerom,kopidato,områdeid,lokalid,fylkeid,komid,ar50_bonitet
0,1,arealressursflate,60,13,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,1,50,5022,11
1,2,arealressursflate,60,11,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,2,50,5022,11
2,3,arealressursflate,60,13,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,3,50,5022,11
3,4,arealressursflate,60,11,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,4,50,5022,11
4,5,arealressursflate,30,13,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,5,50,5022,4


In [19]:
# lookup table 
lookup_csv = os.path.join(data_path, "AR50_bonitet_lookup.csv")
lookup_df = pd.read_csv(lookup_csv)

# rename cols to correspond with AR50 lyr
lookup_df.rename(
    columns={
        "ARTYPE kode": "artype",
        "ARTRESLAG kode": "artreslag",
        "ARSKOGBON kode": "arskogbon",
        "ARJORDBR kode": "arjordbr",
        "ARDYRKING kode": "ardyrking",
        "ARVEGET kode": "arveget",
        "Bonitet kode": "ar50_bonitet",
    },
    inplace=True,
)

# reorder cols to correspond with AR50 lyr
lookup_df = lookup_df[
    [
        "artype",
        "arskogbon",
        "artreslag",
        "arveget",
        "arjordbr",
        "ardyrking",
        "Beskrivelse",
        "ar50_bonitet",
    ]
]

display(lookup_df.head(11))

,artype,arskogbon,artreslag,arveget,arjordbr,ardyrking,Beskrivelse,ar50_bonitet
0,10,98,98,98,98,98,Bebygd og samferdsel,14
1,20,98,98,98,24,98,Fulldyrka og Overflatedyrka jord,1
2,20,98,98,98,25,82,"Innmarksbeite, dyrkbar jord",2
3,20,98,98,98,25,81,Innmarksbeite,2
4,20,98,98,98,99,99,"Jordbruksareal, uspesifisert",1
5,30,11,31,98,98,82,"Barskog, impediment, dyrkbar jord",6
6,30,11,31,98,98,81,"Barskog, impediment",6
7,30,12,31,98,98,82,"Barskog, lav skogbonitet, dyrkbar jord",5
8,30,12,31,98,98,81,"Barskog, lav skogbonitet",5
9,30,13,31,98,98,82,"Barskog, middels skogbonitet, dyrkbar jord",4


In [20]:
# create lookup dict
# keys must be in same order as gpkg lyr fields
keys = (
    "artype",
    "arskogbon",
    "artreslag",
    "arveget",
    "arjordbr",
    "ardyrking",
    )

value= "ar50_bonitet"

lookup_dict = create_lookup_dict(
    lookup_df,
    keys=keys,
    value=value
)

# print first two entries of dict
print({k: lookup_dict[k] for k in list(lookup_dict)[:11]})

{(10, 98, 98, 98, 98, 98): 14, (20, 98, 98, 98, 24, 98): 1, (20, 98, 98, 98, 25, 82): 2, (20, 98, 98, 98, 25, 81): 2, (20, 98, 98, 98, 99, 99): 1, (30, 11, 31, 98, 98, 82): 6, (30, 11, 31, 98, 98, 81): 6, (30, 12, 31, 98, 98, 82): 5, (30, 12, 31, 98, 98, 81): 5, (30, 13, 31, 98, 98, 82): 4, (30, 13, 31, 98, 98, 81): 4}


In [21]:
if False:
    # loop through the features and reclassify the attribute value "ar50_bonitet"
    features_to_update = []

    for feature in lyr:
        # get the attribute values
        artype = feature.GetField("artype")
        arskogbon = feature.GetField("arskogbon")
        artreslag = feature.GetField("artreslag")
        arveget = feature.GetField("arveget")
        arjordbr = feature.GetField("arjordbr")
        ardyrking = feature.GetField("ardyrking")

        key = (int(artype), int(arskogbon), int(artreslag), int(arveget), int(arjordbr), int(ardyrking))
        if key in lookup_dict:
            new_value = lookup_dict[key]
            feature.SetField("ar50_bonitet", new_value)
            features_to_update.append(feature)

    print(f"Number of features to update: {len(features_to_update)}")
    # Batch update features
    for feature in features_to_update:
        lyr.SetFeature(feature)

    print("Finished updating Bonitet.")

    df_AR50_bon = pd.DataFrame(feature.items() for feature in islice(lyr, 5))
    display(df_AR50_bon.head())

# close OGR object
ds = None

## 1.3 Load data into Spatial Database (DuckDB)


### Load data into DuckDB

In [ ]:
if test_area:
    db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
else:
    db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")

In [ ]:
print(db_path)

**Load Bioklima soner into DuckDB**

In [ ]:
if prepare_duckdb:
    # AR50 into duckdb
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "bioklima_2017_coastalclip")
    else:
        in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "bioklima_2017_coastalclip")

**Load AR50 into DuckDB**

In [ ]:
# run time: 
if prepare_duckdb:
    # AR50 into duckdb
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "ar50_flate")
    else:
        in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "ar50_flate")

**Load Protected Areas into DuckDB**

In [ ]:
# import protected areas into DuckDB

# if Test is True, import test gpkg otherwise import the original gpkg
if prepare_duckdb:
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "vern_25833_bbox.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "naturvernomrade")
        load_gpkg_layers(db_path, in_gpkg, "foreslatt_vern")
    else:
        in_gpkg = os.path.join(data_path, "interim", "vern_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "naturvernomrade")
        load_gpkg_layers(db_path, in_gpkg, "foreslatt_vern")

**Load Regions into DuckDB**

In [ ]:
if prepare_duckdb:
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "admin_25833_bbox.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "region_2024")
        load_gpkg_layers(db_path, in_gpkg, "norge_landareal_n50")
        load_gpkg_layers(db_path, in_gpkg, "norge_hav_land_n50")
        load_gpkg_layers(db_path, in_gpkg, "norge")
    else:
        in_gpkg = os.path.join(data_path, "interim", "admin_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "region_2024")
        load_gpkg_layers(db_path, in_gpkg, "norge_landareal_n50")
        load_gpkg_layers(db_path, in_gpkg, "norge_hav_land_n50")
        load_gpkg_layers(db_path, in_gpkg, "norge")

**Load SSB 10km Grid into DuckDB**

In [ ]:
# import SSB 10km grid

if prepare_duckdb:
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "admin_25833_bbox.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "norge_ruter_10km")
    else:
        in_gpkg = os.path.join(data_path, "interim", "admin_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "norge_ruter_10km")

### Preprocess DuckDB Tables 

**Convert BLOB columns to GEOM columns**

In [ ]:
if prepare_duckdb:
    # cast BLOB (Binary Large Object) to geometry for spatial operations
    
    import duckdb

    con = duckdb.connect(db_path)
    con.install_extension('spatial')
    con.load_extension('spatial')
    
    # duckdb tables names to list
    tables = con.execute("SHOW TABLES;").fetchdf()
    tables = tables["name"].to_list()
    print(tables)
    
    for table in tables:
        
        blob_to_geom(
            db_path=db_path,
            tbl_name=table,
            blob_field="geometry",
            geom_field="geom",   
        )

### Display DuckDB tables 

In [25]:
import duckdb

con = duckdb.connect(db_path)
con.install_extension('spatial')
con.load_extension('spatial')

In [24]:
reset_method =True 

tables_to_keep = [
    "ar50_flate", 
    "naturvernomrade", 
    "foreslatt_vern",
    "naturvernomrade_infra", 
    "foreslatt_vern_infra", 
    "region_2024", 
    "norge_landareal_n50",
    "norge_hav_land_n50",
    "norge",
    "norge_ruter_10km", 
    "bioklima_2017_coastalclip"
    ]

all_tables = [item[0] for item in con.execute("SHOW TABLES;").fetchall()]
print(all_tables)

if reset_method:
    # Iterate over all tables and remove tables that are not in the list of tables to keep
    [remove_table(db_path, tbl) for tbl in all_tables if tbl not in tables_to_keep]

# print tables
con.sql("SHOW TABLES;")

[]


┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘

In [ ]:
# display first 5 rows of the table
table = "foreslatt_vern"
con.sql(f"SELECT * FROM {table} LIMIT 5")

In [ ]:
# close
con.close()

## 1.4 Display Data on the Map

https://ipyleaflet.readthedocs.io/en/latest/installation/index.html#using-pip 


In [ ]:
# Display data on map using leafmap
# TODO display duckdb tables as well, not only WMS

init_location = [62.223207, 9.550195]  # Hjerkinn
zoom_start = 13  
min_zoom = 8  # keeps user from zooming out too far
basemap = leafmap.Map(
    location=init_location, 
    zoom=zoom_start, 
    min_zoom=min_zoom, 
    max_bounds=True
    )

# set background
basemap.add_basemap("SATELLITE", opacity=0.7)

# add WMS verneområder
wms_url ="https://kart.miljodirektoratet.no/arcgis/services/vern/mapserver/WMSServer"
wms_layer = "naturvern_klasser_omrade"
wms_name = "Protected Areas"
basemap.add_wms_layer(
    url=wms_url, 
    layers=wms_layer, 
    name=wms_name, 
    wms_format="image/png",
    )

if test_area:
    # add bounding box - Hjerkinn
    basemap.add_gdf(gdf_BB, layer_name="Test Area", fill_color="blue", fill_opacity=0.2, weight=2)

basemap

# 2. Methods

## 2.1 Overlay Analysis

In [ ]:
if statistical_zone in ["norge_landareal_n50", "region_2024", "norge"]:
    # add a column "identifikasjon_lokalId" to the table "norge_landareal_n50"
    # fill column 0 - x
    con = duckdb.connect(db_path)
    con.install_extension('spatial')
    con.load_extension('spatial')
    
    con.execute(f"ALTER TABLE {statistical_zone} ADD COLUMN identifikasjon_lokalId INTEGER;")
    con.execute(f"UPDATE {statistical_zone}  SET identifikasjon_lokalId = rowid;")
    con.close()
    print(statistical_zone)


### Raster overlay | Calculate infrastrukturindeks statistics

| Protected area | min |  max | mean | std | median |
|----------------|-----|------|------|-----|--------|
| Area  A        |     |      |      |     |        |

In [ ]:
# table to df 

# to dataframe
con = duckdb.connect(db_path)
con.install_extension('spatial')
con.load_extension('spatial')

df = con.execute(f"SELECT ST_AsText(geom) as geometry, * FROM {statistical_zone}").fetchdf()

# drop "geom"
df = df.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df['geometry'] = df['geometry'].apply(wkt.loads)

# to gpd
gdf = gpd.GeoDataFrame(df, geometry='geometry')
print(f"Number of unique ids: {gdf['identifikasjon_lokalId'].nunique()}")
print(f"Number of rows: {gdf.shape[0]}")

# drop all rows where id is NULL
gdf = gdf.dropna(subset=["identifikasjon_lokalId"])

display(gdf.head())
con.close()

In [ ]:
if run_infra:
    from py_scripts.raster import * 

    # calculate the min, max, mean, std and median of the infrastruktur raster 
    # within the protected areas with a buffer of 1000m
    raster_path = os.path.join(interim_data, "infra_25m_25833_coastalclip.tif")

    if statistical_zone == "norge":
        buffer_distance = 0
    elif statistical_zone == "norge_landareal_n50":
        buffer_distance = 0
    elif statistical_zone == "region_2024":
        buffer_distance = 0
    else:
        buffer_distance = 1000
    print(buffer_distance)
    
    gdf_infra = overlay_stats(raster_path, gdf, buffer_distance=buffer_distance)
    
    # remove table study area
    remove_table(db_path, str(statistical_zone))
    display(gdf_infra.head())

In [ ]:
con = duckdb.connect(db_path)
# print tables
con.sql("SHOW TABLES;")

In [ ]:
con.close()

In [ ]:
# export to gpkg interim
out_gpkg = os.path.join(interim_data, "interim_25833.gpkg")
out_layer = f"{statistical_zone}_infra"
# export to specifc layer "stastiscial_zone_infra"

gdf_infra.crs = "EPSG:25833"
# Write to existing .geopackage
gdf_infra.to_file(out_gpkg, driver='GPKG', layer=out_layer, mode='w')

In [ ]:
# to duckdb
load_gpkg_layers(db_path, out_gpkg, out_layer)

# convert geom
blob_to_geom(
    db_path=db_path,
    tbl_name=out_layer,
    blob_field="geometry",
    geom_field="geom",   
)



### Vector overlay | Bioclimatic Zones
**Bioklimatiske soner**

| Protected area | Sone 1 (%) | Sone 2 (%) | Sone 3 (%) | ... |
|----------------|------------|------------|------------|-----|
| *NaturvernId*  |*bioklima_1*|*bioklima_2*|*bioklima_3*|...  |
| area A         |            |            |            |     |

In [ ]:
if run_bioklima:
    tbl_study_area = f"{statistical_zone}_infra"
    id = "identifikasjon_lokalId"
    tbl_bioklima = "bioklima_2017_coastalclip"
    bioklima_field = "Sone_Kode"
    bioklima_zones = ["6SO-1", "6SO-2", "6SO-3", "6SO-4", "6SO-5"]
    for area_class in bioklima_zones: # 1-18
        print(f"Calculating Bioklima class {area_class}")
        new_field = f"s_{area_class}_m2" # ar50_bon1
        new_field = new_field.replace("-", "_")
        remove_field(db_path, tbl_study_area, new_field)
        bioklima_area_class(db_path, tbl_study_area, id, tbl_bioklima, bioklima_field, area_class, new_field)
    
    for area_class in bioklima_zones: # 1-18
        new_field = f"s_{area_class}_m2" # ar50_bon1
        new_field = new_field.replace("-", "_")
        con = duckdb.connect(db_path)
        con.install_extension('spatial')
        con.load_extension('spatial')
        con.sql(f"UPDATE {tbl_study_area} SET {new_field} = 0 WHERE {new_field} IS NULL")
        con.close()

In [ ]:
con = duckdb.connect(db_path)
con.sql(f"SELECT * FROM {tbl_study_area} LIMIT 5")

### Vector overlay | AR50 (land cover)

**Divide the protected areas into terrestrial and marine areas**

AR50 polygons with classified as "Hav" are considered marine areas, all other areas are considered terrestrial.
- Bonitet = 17 (Hav) 
- Arealtype = 82 (Hav)

<br>

**AR50 - Bonitet**
Calculated area overlap with AR50 Bonitet classes. 

| Protected area | Fulldyrka og overflatedyrka jord (m2) | Innmarksbeite (m2) | Skog, høg og særs høg bonitet (m2) | ... |
|----------------|--------------------------------------|-------------------|-----------------------------------|-----|
| *NaturvernId*  |*AR50_bon_1*|*AR50_bon2*|*AR50_bon3*|...|
| area A         |                                      |                   |                                   |     |


In [ ]:
# calculate ar50 area overlapp with protected areas
tbl_study_area = f"{statistical_zone}_infra"
id = "identifikasjon_lokalId"
tbl_ar50 = "ar50_flate"
ar50_field = "ar50_bonitet"

if run_ar50:
    for area_class in range(1, 19): # 1-18
        print(f"Calculating Bonitet class {area_class}")
        new_field = f"ar50_bon{area_class}_m2" # ar50_bon1
        remove_field(db_path, tbl_study_area, new_field)
        ar50_area_class(db_path, tbl_study_area, id, tbl_ar50, ar50_field, area_class, new_field)

    # if ar50_bonx is null, set to 0
    for area_class in range(1, 19): # 1-18
        new_field = f"ar50_bon{area_class}_m2" # ar50_bon1
        con = duckdb.connect(db_path)
        con.install_extension('spatial')
        con.load_extension('spatial')
        con.sql(f"UPDATE {tbl_study_area} SET {new_field} = 0 WHERE {new_field} IS NULL")
        con.close()

In [ ]:
# print first 5 rows of the table statistical_zone
con = duckdb.connect(db_path)
con.sql(f"SELECT * FROM {tbl_study_area} LIMIT 5")

# print as dataframe
df = con.execute(f"SELECT * FROM {tbl_study_area} LIMIT 5").fetchdf()

# display columns: identfifikasjon_lokalId, ar50_bonx
col_id= [id]
cols_ar50 = [col for col in df.columns if "ar50_bon" in col and col != "sum_ar50_bon_m2"]
cols = col_id + cols_ar50
df_ar50 = df[cols]
display(df)

# close
con.close()

**Check if sum of all AR50 classes correspons with the total area**

In [ ]:
# Calculating sum of all bonitet classes
remove_field(db_path, tbl_study_area, "sum_ar50_bon_m2")
sum_area_cols(db_path, tbl_study_area, id, cols_ar50, "sum_ar50_bon_m2")

# Calculating area of each polygon in the study area
remove_field(db_path, tbl_study_area, "areal_m2")
geom_area(db_path, tbl_study_area, "geom", "areal_m2")

# Calculateing the difference between the sum of all bonitet classes and the area of each polygon
remove_field(db_path, tbl_study_area, "area_diff_m2")
area_difference(db_path, tbl_study_area, "sum_ar50_bon_m2", "areal_m2", "area_diff_m2")

In [ ]:
# print first 5 rows of the table statistical_zone
con = duckdb.connect(db_path)

# print as dataframe
df = con.execute(f"SELECT * FROM {tbl_study_area}").fetchdf()

# display columns: identfifikasjon_lokalId, ar50_bonx
col_id= [id]
cols = col_id + ["sum_ar50_bon_m2", "areal_m2", "area_diff_m2"]
df_ar50 = df[cols]

# sort by area_diff_m2
df_ar50 = df_ar50.sort_values(by="area_diff_m2", ascending=True)
display(df_ar50)

# close
con.close()

In [ ]:
# print number of rows 
print(f"Number of rows: {df.shape[0]}")

# print number of unique ids "identifikasjon_lokalId"
print(f"Number of unique ids: {df['identifikasjon_lokalId'].nunique()}")

# print number of rows with value in "ar50_bon17"
print(f"Number of areas with Marine area: {df['ar50_bon17_m2'].count()}")

## 2.2 Spatial indices


### Shape Index

| Protected area | Total area (km2) | Perimeter (km) | Land area (km2) | Perimeter land area (km) |
|----------------|------------|----------------|-----------------|-------------------------|
| *NaturvernId* | *areal_m2* | *omkrets_m* | *landareal_m2* | *landomkrets_m2*|
| area A         |    *        |                |                 |                         |

***
$\mathbf{\text{Shape Index}}$<br>
***
The shape index is a measure of how compact the shape is compared to a circle with the same area. The shape index is calculated for the entire protected area and the land area.

- $P$ = perimeter
- $r$ = radius
- $A$ = area


**Shape Index:**&emsp;      $SI = \frac{P}{2\pi r}$

**Radius:**&emsp; $r = \sqrt{\frac{A}{\pi}}$

- SI = 1, shape is a perfect circle
- SI > 1, shape is less compact than a circle
- SI < 1, is not possible 

In [ ]:
tbl_havland = "norge_hav_land_n50"

In [ ]:
if statistical_zone == "norge" or statistical_zone == "region_2024":
    # print first 5 rows of the table statistical_zone
    con = duckdb.connect(db_path)
    #spatial
    con.install_extension('spatial')
    con.load_extension('spatial')
    
    # to dataframe
    df = con.execute(f"SELECT ST_AsText(geom) as geometry, * FROM {tbl_study_area}").fetchdf()

    # drop "geom"
    df = df.drop(columns=["geom"])

    # Convert the geom column to gpd GeoDataFrame
    df['geometry'] = df['geometry'].apply(wkt.loads)

    # to gpd
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    print(f"Number of unique ids: {gdf['identifikasjon_lokalId'].nunique()}")


    display(gdf)
        
    # quite notebook
    # Export GDF to file 
    filepath = os.path.join(data_path, "processed", "vern_og_bevaring")
    gdf.crs = "EPSG:25833"
    # Write to existing .geopackage
    gdf.to_file(os.path.join(filepath + '.gpkg'), driver='GPKG', layer=f'{statistical_zone}_overlapp', mode='w')

    # gdf to df and remove geom
    df = gdf.drop(columns=["geometry"])
    # Write to .csv
    df.to_csv(os.path.join(data_path, "processed", f'{statistical_zone}_overlapp.csv'))
    quit()

In [ ]:
from py_scripts.my_duckdb import *

In [ ]:
if run_shape_index:
    # create two new tables with only terrestrial part and marine part, respectively.
    remove_table(db_path, sz_land)
    remove_table(db_path, sz_sea)
    remove_table(db_path, f"{sz_land}_tmp1")
    remove_table(db_path, f"{sz_sea}_tmp1")
    extract_overlap_geom(db_path, id, tbl_havland, "objtype", "hav", tbl_study_area, f"{sz_sea}_tmp1", f"{sz_land}_tmp1")

In [ ]:
if run_shape_index:

    # remove slivers (lines/points) from the tables
    remove_table(db_path, f"{sz_land}_tmp2")
    remove_table(db_path, f"{sz_sea}_tmp2")
    delete_lines_points(db_path, f"{sz_land}_tmp1", f"{sz_land}_tmp2")
    delete_lines_points(db_path, f"{sz_sea}_tmp1", f"{sz_sea}_tmp2")

    # to multipolygon
    remove_table(db_path, sz_land)
    remove_table(db_path, sz_sea)
    group_to_multipolygon(db_path, f"{sz_land}_tmp2", sz_land, id)
    group_to_multipolygon(db_path, f"{sz_sea}_tmp2", sz_sea, id)

In [ ]:
dict_geom = {
    tbl_study_area: ("areal_m2", "omkrets_m", "formindeks"),
    sz_land: ("landareal_m2", "landomkrets_m", "land_formindeks"),
    sz_sea: ("sjoareal_m2", "sjoomkrets_m", "sjo_formindeks"),
}

for key, value in dict_geom.items():
    
    # areal
    #remove_field(db_path, key, "areal_not_grouped")
    #geom_area(db_path, key, "geom", "areal_not_grouped")
    
    # areal by ID
    remove_field(db_path, key, value[0])
    geom_area_byID(db_path, key, "identifikasjon_lokalId", "geom", value[0])
    
    # omkrets
    #remove_field(db_path, key, "omkrets_not_grouped")
    #geom_peri(db_path, key, "geom", "omkrets_not_grouped")
    
    # omkrets by ID
    remove_field(db_path, key, value[1])
    geom_peri_byID(db_path, key, "identifikasjon_lokalId", "geom", value[1])
    
    # formindeks
    #remove_field(db_path, key, "formindeks_not_grouped")
    #geom_index(db_path, key, "geom", "formindeks_not_grouped")
    
    # formindeks by ID
    remove_field(db_path, key, value[2])
    geom_index_byID(db_path, key, "identifikasjon_lokalId", "geom", value[2])


In [ ]:
con = duckdb.connect(db_path)
con.install_extension('spatial')
con.load_extension('spatial')

In [ ]:
con.sql("SHOW TABLES;")

In [ ]:
con.sql(f"SELECT * FROM {sz_land} LIMIT 5")

In [ ]:
con.sql(f"SELECT * FROM {sz_sea} LIMIT 5")

**Export to Land and Sea Geometories to GDF and GPKG-layer**

In [ ]:
# to dataframe
df_land = con.execute(f"SELECT ST_AsText(geom) as geometry, * FROM {sz_land}").fetchdf()

# drop "geom"
df_land = df_land.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df_land['geometry'] = df_land['geometry'].apply(wkt.loads)

# to gpd
gdf_land = gpd.GeoDataFrame(df_land, geometry='geometry')
print(f"Number of unique ids: {df_land['identifikasjon_lokalId'].nunique()}")

display(gdf_land.head())

# Export GDF to file 
filepath = os.path.join(data_path, "processed", "vern_og_bevaring")
gdf_land.crs = "EPSG:25833"
# Write to existing .geopackage
gdf_land.to_file(os.path.join(filepath + '.gpkg'), driver='GPKG', layer=sz_land, mode='w')


In [ ]:
# to dataframe
df_sjo = con.execute(f"""
    SELECT ST_AsText(geom) as geometry, * 
    FROM {sz_sea}
    WHERE ST_GeometryType(geom) IN ('POLYGON', 'MULTIPOLYGON')
""").fetchdf()

# delete POLYGON EMPTY
df_sjo = df_sjo[df_sjo['geometry'] != "POLYGON EMPTY"]

# drop "geom"
df_sjo = df_sjo.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df_sjo['geometry'] = df_sjo['geometry'].apply(wkt.loads)

# to gpd
gdf_sjo = gpd.GeoDataFrame(df_sjo, geometry='geometry')
print(f"Number of unique ids: {gdf_sjo['identifikasjon_lokalId'].nunique()}")

display(gdf_sjo)

# Export GDF to file 
filepath = os.path.join(data_path, "processed", "vern_og_bevaring")
gdf_sjo.crs = "EPSG:25833"
# Write to existing .geopackage
gdf_sjo.to_file(os.path.join(filepath + '.gpkg'), driver='GPKG', layer=sz_sea, mode='w')

In [ ]:
# drop geometry column in duckdb and keep only unique ids in table

# drop geom column
remove_field(db_path, sz_land, "geom")
remove_field(db_path, sz_sea, "geom")

# drop duplicates 
remove_duplicates(db_path, sz_land, "identifikasjon_lokalId")
remove_duplicates(db_path, sz_sea, "identifikasjon_lokalId")

In [ ]:
# number of rows in the new table
con.sql(f"SELECT COUNT(*) FROM {sz_land}")

In [ ]:
# number of unique ids in the new table
con.sql(f"SELECT COUNT(DISTINCT identifikasjon_lokalId) FROM {sz_land}")

In [ ]:
remove_table(db_path, sz_land_sea)
join_tables_create_new(
    db_path, 
    tbl1=sz_land,
    tbl2=sz_sea,
    id_field="identifikasjon_lokalId",
    new_tbl=sz_land_sea
)

remove_field(db_path, sz_land_sea, "identifikasjon_lokalId_1") 
con.sql("SHOW TABLES;")

In [ ]:
con.sql(f"SELECT * FROM {sz_land_sea} LIMIT 5")

In [ ]:
# number of rows in the new table
con.sql(f"SELECT COUNT(*) FROM {sz_land_sea}")

In [ ]:
# number of unique ids in the new table
con.sql(f"SELECT COUNT(DISTINCT identifikasjon_lokalId) FROM {sz_land_sea}")

In [ ]:
remove_table(db_path, sz_land_vars)
join_tables_create_new(
    db_path, 
    tbl1=tbl_study_area,
    tbl2=sz_land_sea,
    id_field="identifikasjon_lokalId",
    new_tbl=sz_land_vars
)

In [ ]:
con.sql("SHOW TABLES;")

In [ ]:
con.sql(f"SELECT COUNT(DISTINCT identifikasjon_lokalId) FROM {sz_land_vars}")

### Density of protected areas 

**Rutenettstatistikk (10x10 km2):**

| SSB grid cell | area | Density (land + marine) | Density (land) | 
| --------------|------|-------------------------|----------------|
| *grid_ID* | *grid_areal* | *tetthet_tot_vern* | *tetthet_landvern* | 
| cell 1| 100 km2 | ||





***
$\mathbf{\text{Density}}$<br>
***
The density of protected area per 10x10km (SSB grid) is calculated by dividing the area of the protected area by 100 km^2.

**Density:**&emsp;      $Density = \frac{A}{100 km^2}$



In [ ]:
# Select by region
# SUM protected area in region 
# Area % = SUM(statistical_zone_m2)/(areal_m2) * 100

# 3. Export Results 
**Clean table**

In [ ]:
# if null set to 0
con.sql(f"""
    UPDATE {sz_land_vars}
    SET sjoareal_m2 = 0, sjoomkrets_m = 0
    WHERE sjoareal_m2 IS NULL
""")

# if null set to 0
con.sql(f"""
    UPDATE {sz_land_vars}
    SET landareal_m2 = 0, landomkrets_m = 0
    WHERE landareal_m2 IS NULL
""")

# remove cols
remove_field(db_path, sz_land_vars, "identifikasjon_lokalId_1")

# print first 5 rows of the table sz_land_vars
con.sql(f"SELECT * FROM {sz_land_vars} LIMIT 5")

In [ ]:
# to dataframe
df = con.execute(f"SELECT ST_AsText(geom) as geometry, * FROM {sz_land_vars}").fetchdf()

# drop "geom"
df = df.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df['geometry'] = df['geometry'].apply(wkt.loads)

# to gpd
gdf = gpd.GeoDataFrame(df, geometry='geometry')
print(f"Number of unique ids: {gdf['identifikasjon_lokalId'].nunique()}")


display(gdf)

In [ ]:
# Export GDF to file 
filepath = os.path.join(data_path, "processed", "vern_og_bevaring")
gdf.crs = "EPSG:25833"
# Write to existing .geopackage
gdf.to_file(os.path.join(filepath + '.gpkg'), driver='GPKG', layer=sz_land_vars, mode='w')


In [ ]:
# gdf to df and remove geom
df = gdf.drop(columns=["geometry"])
# Write to .csv
df.to_csv(os.path.join(data_path, "processed", f'{statistical_zone}_overlapp.csv'))

In [ ]:
quit()

**Remove temporary tables in duckDB**

In [ ]:
tables = ["vern_land_tmp1", "vern_land_tmp2", "vern_sjo_tmp1", "vern_sjo_tmp2"]
#[remove_table(db_path, tbl) for tbl in tables]